In [14]:
import pandas as pd


%store -r dm
dm = dm

In [15]:
data_A = dm.data_A
data_A


,pv_measurement,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,0.0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,...,342.834015,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0
1,0.0,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,...,346.294006,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0
2,0.0,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,...,349.768005,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0
3,0.0,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,...,353.251007,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0
4,0.0,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,...,356.742004,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136237,0.0,2023-04-30 22:00:00,4.5,1.281,1532.900024,0.0,0.0,519.000000,0.0,272.299988,...,341.851013,-10.630,0.1,274.399994,97.400002,16674.900391,4.3,3.4,2.6,-0.0
136238,0.0,2023-04-30 22:15:00,4.5,1.281,1413.199951,0.0,0.0,524.900024,0.0,272.299988,...,345.524994,-11.102,0.1,274.399994,98.699997,15005.000000,4.2,3.3,2.6,-0.0
136239,0.0,2023-04-30 22:30:00,4.5,1.281,1293.599976,0.0,0.0,530.700012,0.0,272.299988,...,349.216003,-11.470,0.1,274.299988,99.500000,13323.200195,4.0,3.1,2.5,-0.0
136240,0.0,2023-04-30 22:45:00,4.5,1.281,1173.900024,0.0,0.0,536.500000,0.0,272.299988,...,352.920013,-11.731,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0


In [16]:
from catboost import CatBoostRegressor

catModel = CatBoostRegressor(verbose=False)

In [17]:

import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Setting date_forecast as index
data_A.set_index('date_forecast', inplace=True)

# Splitting data into X and y
X = data_A.drop(columns=['pv_measurement'])
y = data_A['pv_measurement']

# Sliding window cross-validation
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    catModel.fit(X_train, y_train)
    predictions = catModel.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f'Mean Squared Error for split: {mse}')
    


Mean Squared Error for split: 395593.9386255842
Mean Squared Error for split: 133413.32173792896
Mean Squared Error for split: 225591.8537480906
Mean Squared Error for split: 221956.03867331197
Mean Squared Error for split: 107200.834297079
